# Hypertrophic Cardiomyopathy Genes Cross-Validation
##### Selin Kubali
##### 12/13/2023
## Goal
Find out whether we can distinguish the HCM risk of bottom 25% and top 25% of missense and deleterious variant carriers in key hypertrophic cardiomyopathy-related genes.

#### How the code functions
Use cross-validation to fit a Cox-PH model and predict hazard scores. Then isolate the bottom 25% and top 25% of carriers by hazard score and calculate whether there is a statistically significant difference in HCM between them use the Mann-Whitney U test.

Cross-validation is done by splitting on variant data, to ensure there are an equal number of variants in each fold and prevent overfitting on high-frequency variants.

#### Inputs
Lifelines files - from running generate_extracts_gnomAD.ipynb on UKBiobank in Cassa Lab Shared Project/selected_genes/hcm/notebooks. Stored in Cassa Lab Shared Project/selected_genes/hcm/lifelines_data. 
Variant data files - from running vep_processing.ipynb on UKBiobank in Cassa Lab Shared Project/selected_genes/hcm/notebooks. Stored in Cassa Lab Shared Project/selected_genes/hcm/parsed_vep_files

#### Note
Two HCM related genes - DES and PLN - were eliminated for having too few variants to converge.
PTPN11, TNNI3, and TTR each have few cases of HCM with missense or deleterious variants, which may harm convergence.

In [1]:
import pandas as pd
import numpy as np
from lifelines import CoxPHFitter
from sklearn.model_selection import KFold
from statsmodels.stats.multitest import multipletests
import warnings; warnings.simplefilter('ignore')
import matplotlib.pyplot as plt 
import numpy as np 

In [57]:
def cross_val(gene):
        cph = CoxPHFitter(penalizer=0.0000001)
        # load lifelines file
        file_name=gene+'.csv'
        lifelines_data = pd.read_csv("/Users/uriel/Downloads/work_temp/cross_val_lifelines/"+file_name, dtype={
                'is_family_hist':'boolean',
                'is_hcm':'boolean'
                })

        # load variant data file
        file_name=gene+'.csv'
        variant_data = pd.read_csv("/Users/uriel/Downloads/work_temp/variant_files/"+file_name)


        clinvar_df = pd.read_csv("/Users/uriel/Downloads/work_temp/variant_summary.txt", sep = '\t')
        clinvar_df = clinvar_df.rename(columns={"Chromosome": "Chrom", "Start": "Pos", "ReferenceAlleleVCF": "Ref", "AlternateAlleleVCF": "Alt"})
        clinvar_df = clinvar_df[["Name", "Chrom", "Pos", "Ref", "Alt", "ClinicalSignificance"]]
        clinvar_variant_df = pd.merge(variant_data, clinvar_df, how = 'left', on = ['Pos', 'Ref', 'Alt'])
        clinvar_variant_df = clinvar_variant_df.rename(columns={"Name_x": "Name"})
        clinvar_variant_df = clinvar_variant_df[["Name", "ClinicalSignificance"]]
        lifelines_data_with_clin_sign = pd.merge(clinvar_variant_df, lifelines_data, how="right", on = ['Name'])
        lifelines_data_with_clin_sign = lifelines_data_with_clin_sign.drop(['Name', 'Carrier', 'index', 'am_pathogenicity'], axis = 1)



        variant_data = variant_data[['Name']]
        variant_data['var_index'] = variant_data.index



        # set lifelines data index to variant data index
        lifelines_data = variant_data.merge(lifelines_data, how="outer")
        lifelines_data.set_index("var_index")

        # clean lifelines file; set pathogenicity for deleterious variants to 1
        lifelines_data.loc[lifelines_data['deleterious'] == 1, 'am_pathogenicity'] = 1
        lifelines_data = lifelines_data.drop(['Name','Carrier', 'index', 'am_pathogenicity'], axis = 1)
        #lifelines_data = lifelines_data[(lifelines_data['deleterious'] == True) | (lifelines_data['missense_variant'] == True)]
        #lifelines_data = lifelines_data.drop(['deleterious','missense_variant', 'synonymous_variant'], axis = 1)

        lifelines_data = lifelines_data.dropna()







        # cross validation: split up phenotypic data file based on variant file index
        kf = KFold(n_splits=5, shuffle=True, random_state=1)
        testing_set = []
        for train_idx, test_idx in kf.split(variant_data):
                train = lifelines_data[lifelines_data['var_index'].isin(train_idx)]
                test = lifelines_data[lifelines_data['var_index'].isin(test_idx)]

                train = train.drop(['var_index'], axis=1)
                test = test.drop(['var_index'], axis=1)

                # fit CPH and add hazard scores

                
                cph.fit(train, duration_col="duration", event_col="is_hcm", fit_options = {"step_size":0.1})
                hazard_scores_fold = cph.predict_partial_hazard(test)
                test['hazard'] = hazard_scores_fold
                testing_set.append(test)



        # create new lifelines_data df by joining all testing sets
        lifelines_data = pd.concat([df for idx, df in enumerate(testing_set)]) 
        lifelines_data = pd.merge(lifelines_data, lifelines_data_with_clin_sign, how = 'left')

        


        


        return lifelines_data
    

In [60]:

def find_params(gene):
        thresholds_list =  list(range(1, 101))



        p_vals = {}
        hazard_ratios = {}
        f1_scores = {}

        
        lifelines_data = cross_val(gene)
        




        # filter for patients with lowest 25% and highest 25% hazard scores

        for i in thresholds_list:
                percentiles = np.percentile(lifelines_data['hazard'], [i])
                bottom = lifelines_data[lifelines_data['hazard'] < percentiles[0]]
                top = lifelines_data[lifelines_data['hazard'] >= percentiles[0]]

                bottom.loc[:,'is_hcm'] = np.where(((bottom['ClinicalSignificance'] == 'Pathogenic') |(bottom['ClinicalSignificance'] == 'Likely pathogenic') | (bottom['ClinicalSignificance'] == 'Pathogenic/Likely pathogenic')) , 1, 0)
                top.loc[:,'is_hcm'] = np.where(((top['ClinicalSignificance'] == 'Pathogenic') |(top['ClinicalSignificance'] == 'Likely pathogenic') | (top['ClinicalSignificance'] == 'Pathogenic/Likely pathogenic')), 1, 0)
                dfA = pd.DataFrame({'E': bottom['is_hcm'], 'T': bottom['duration'], 'is_highest': 0})
                dfB = pd.DataFrame({'E': top['is_hcm'], 'T': top['duration'], 'is_highest': 1})


                """bottom.loc[:,'is_hcm'] = np.where(bottom['is_hcm'] == True, 1, 0)
                top.loc[:,'is_hcm'] = np.where(top['is_hcm'] == True, 1, 0)
                dfA = pd.DataFrame({'E': bottom['is_hcm'], 'T': bottom['duration'], 'is_highest': 0})
                dfB = pd.DataFrame({'E': top['is_hcm'], 'T': top['duration'], 'is_highest': 1})"""


                df = pd.concat([dfA, dfB])

                cph = CoxPHFitter().fit(df, 'T', 'E', fit_options = {"step_size":0.1})
                hazard_ratios.update({i:cph.hazard_ratios_.at['is_highest']})

                p_vals.update({i:cph.summary['p'].at['is_highest']})

                TP = (((top['ClinicalSignificance'] == 'Pathogenic') | (top['ClinicalSignificance'] == 'Likely pathogenic') | (top['ClinicalSignificance'] == 'Pathogenic/Likely pathogenic')).sum())
                FN = (((bottom['ClinicalSignificance'] == 'Pathogenic') | (bottom['ClinicalSignificance'] == 'Likely pathogenic') | (bottom['ClinicalSignificance'] == 'Pathogenic/Likely pathogenic')).sum())
                #FP = (top['synonymous_variant'].sum())
                FP = (((top['ClinicalSignificance'] == 'Benign') | (top['ClinicalSignificance'] == 'Likely Benign') | (top['ClinicalSignificance'] == 'Benign/Likely benign')).sum())

        
                """
                FN = (bottom['is_hcm'] == 1).sum()
                FP = (top['is_hcm'] == 0).sum()
                TN = (bottom['is_hcm'] == 0).sum()
                """



                
             

                #print('Top HCM',top[top["is_hcm"] == 1]["is_hcm"].sum())
                #print('Bottom HCM', bottom[bottom["is_hcm"] == 1]["is_hcm"].sum())


                print(TP, FN, FP)

                f1_score = TP/(TP+0.5*(FP+FN))
                f1_scores.update({i:f1_score})

                


                # true positive = classified as high risk, ClinVar pathogenic/likely pathogenic
                # false positive = classified as low risk, ClinVar pathogenic 
                # false negative = classified as high risk, synonymous

                # find # of low risk syn vs high risk syn, etc.






        
        return p_vals, hazard_ratios, f1_scores





#### Find thresholds

In [63]:
genes = ['ACTN2', 'ALPK3', 'FLNC','MYBPC3','MYH6', 'MYH7', 'PTPN11', 'TNNI3', 'TTR']
lowest_thresholds_by_p_val = {}
thresholds_by_f1_score = {}
threshold_by_odds_ratio = {}


for gene in genes:
    print(gene)
    p_vals, hazard_ratios, f1_scores = find_params(gene)
    
    # By minimizing p-value
    associated_p_threshold = min(p_vals, key=p_vals.get)
    lowest_thresholds_by_p_val.update({gene:associated_p_threshold})
    associated_f1_threshold = max(f1_scores, key=f1_scores.get)
    thresholds_by_f1_score.update({gene:associated_f1_threshold})

    associated_odds_threshold = max(hazard_ratios, key=hazard_ratios.get)
    threshold_by_odds_ratio.update({gene:associated_odds_threshold})

    



MYH6
2 0 931
2 0 917
2 0 911
2 0 909
2 0 906
2 0 900
2 0 895
2 0 894
2 0 893
2 0 888
2 0 886
2 0 880
2 0 874
2 0 867
2 0 862
2 0 858
2 0 852
1 1 846
1 1 841
1 1 834
1 1 824
1 1 810
1 1 798
1 1 791
1 1 789
1 1 779
1 1 772
1 1 769
1 1 763
1 1 760
1 1 748
1 1 739
1 1 721
1 1 713
1 1 702
1 1 696
1 1 688
1 1 679
1 1 667
1 1 650
1 1 645
1 1 635
1 1 622
1 1 615
1 1 609
1 1 601
1 1 590
1 1 582
1 1 571
1 1 568
1 1 560
1 1 550
1 1 541
1 1 526
1 1 515
1 1 503
1 1 497
1 1 486
1 1 472
1 1 460
1 1 448
1 1 437
1 1 422
1 1 413
1 1 402
1 1 385
1 1 378
1 1 374
1 1 365
1 1 359
1 1 342
1 1 333
1 1 323
1 1 314
1 1 305
1 1 291
0 2 278
0 2 262
0 2 251
0 2 240
0 2 228
0 2 220
0 2 211
0 2 202
0 2 192
0 2 179
0 2 168
0 2 154
0 2 145
0 2 134
0 2 121
0 2 104
0 2 96
0 2 85
0 2 76
0 2 60
0 2 50
0 2 41
0 2 22


ConvergenceError: Convergence halted due to matrix inversion problems. Suspicion is high collinearity. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-modelMatrix is singular.

#### Rerun CoxPH cross-validation with chosen threshold

In [34]:
def find_threshold_vals(dict):

        p_vals = {}
        hazard_ratios = {}
        f1_scores = {}


        # filter for patients with lowest 25% and highest 25% hazard scores

        for gene in dict:
                threshold = dict[gene]


                lifelines_data = cross_val(gene)



                percentiles = np.percentile(lifelines_data['hazard'], [threshold])
                bottom = lifelines_data[lifelines_data['hazard'] < percentiles[0]]
                top = lifelines_data[lifelines_data['hazard'] >= percentiles[0]]
                bottom.loc[:,'is_hcm'] = np.where(bottom['is_hcm'] == True, 1, 0)
                top.loc[:,'is_hcm'] = np.where(top['is_hcm'] == True, 1, 0)

                print(gene)



                dfA = pd.DataFrame({'E': bottom['is_hcm'], 'T': bottom['duration'], 'is_highest': 0})
                dfB = pd.DataFrame({'E': top['is_hcm'], 'T': top['duration'], 'is_highest': 1})
                df = pd.concat([dfA, dfB])

                cph = CoxPHFitter().fit(df, 'T', 'E', fit_options = {"step_size":0.1})
                hazard_ratios.update({gene:cph.hazard_ratios_.at['is_highest']})

                p_vals.update({gene:cph.summary['p'].at['is_highest']})
                p_adjusted = multipletests(list(p_vals.values()), alpha=0.05, method='bonferroni')
                updated_p_dict = {key: new_p_val for key, new_p_val in zip(p_vals.keys(), p_adjusted[1])}

                """TP = (((top['ClinicalSignificance'] == 'Pathogenic') | (top['ClinicalSignificance'] == 'Likely pathogenic') | (top['ClinicalSignificance'] == 'Pathogenic/Likely pathogenic')).sum())
                FN = (((bottom['ClinicalSignificance'] == 'Pathogenic') | (bottom['ClinicalSignificance'] == 'Likely pathogenic') | (bottom['ClinicalSignificance'] == 'Pathogenic/Likely pathogenic')).sum())
                #FP = (top['synonymous_variant'].sum())
                FP = (((top['ClinicalSignificance'] == 'Benign') | (top['ClinicalSignificance'] == 'Likely benign') | (top['ClinicalSignificance'] == 'Benign/Likely benign')).sum())"""

                        
                TP = (top['is_hcm'] == 1).sum()
                FN = (bottom['is_hcm'] == 1).sum()
                FP = (top['is_hcm'] == 0).sum()
             


                f1_score = TP/(TP+0.5*(FP+FN))


                f1_scores.update({gene:f1_score})

               
                

                


        return updated_p_dict, hazard_ratios, f1_scores

#### Convert lowest p-values to dataframe

In [ ]:
updated_p_dict, hazard_ratios, f1_scores = find_threshold_vals(lowest_thresholds_by_p_val)
p_vals = pd.DataFrame.from_dict(updated_p_dict, orient = 'index')
p_vals.columns = ["P-value"]
hazard_ratios = pd.DataFrame.from_dict(hazard_ratios, orient = 'index')
hazard_ratios.columns = ["Odds ratio"]
thresholds = pd.DataFrame.from_dict(lowest_thresholds_by_p_val, orient = 'index')
thresholds.columns = ["Thresholds"]
f1_scores = pd.DataFrame.from_dict(f1_scores, orient = 'index')
f1_scores.columns = ["F1 scores"]
df = p_vals.join(hazard_ratios).join(thresholds).join(f1_scores)
df


ACTN2
ALPK3
FLNC
MYBPC3
MYH6
MYH7
PTPN11
TNNI3
TTR


,P-value,Odds ratio,Thresholds,F1 scores
ACTN2,1.414956e-01,4.063891,91,0.010471
ALPK3,8.145572e-04,3.986677,83,0.010518
FLNC,9.406915e-07,6.811688,83,0.009797
MYBPC3,9.209080e-19,31.108449,98,0.076923
MYH6,2.808500e-08,22.297404,96,0.021239
MYH7,4.972407e-12,7.440186,89,0.034700
PTPN11,1.100783e-01,0.068621,1,0.004496
TNNI3,1.000000e+00,0.334120,15,0.007463
TTR,1.000000e+00,2.256571,53,0.008264


#### Convert highest odds ratios to dataframe

In [107]:
updated_p_dict, hazard_ratios, f1_scores = find_threshold_vals(threshold_by_odds_ratio)
p_vals = pd.DataFrame.from_dict(updated_p_dict, orient = 'index')
p_vals.columns = ["P-value"]
hazard_ratios = pd.DataFrame.from_dict(hazard_ratios, orient = 'index')
hazard_ratios.columns = ["Odds ratio"]
thresholds = pd.DataFrame.from_dict(threshold_by_odds_ratio, orient = 'index')
thresholds.columns = ["Thresholds"]
f1_scores = pd.DataFrame.from_dict(f1_scores, orient = 'index')
f1_scores.columns = ["F1 scores"]
df = p_vals.join(hazard_ratios).join(thresholds).join(f1_scores)
df


      sex  is_family_hist  is_hcm   duration        age  \
0     0.0            True   False  66.368241  66.368241   
1     1.0           False   False  82.535250  82.535250   
2     1.0           False   False  53.535934  53.535934   
3     0.0           False   False  73.618070  73.618070   
4     0.0            True   False  61.535934  61.535934   
...   ...             ...     ...        ...        ...   
8306  1.0            True   False  82.116359  82.116359   
8307  0.0            True   False  75.865845  75.865845   
8308  0.0            True   False  62.866530  62.866530   
8309  1.0           False   False  77.368925  77.368925   
8310  1.0           False   False  71.950719  71.950719   

      principal_component_1  principal_component_4  prs_score  GERP++_RS  trv  \
0                 -10.16980              -3.208120  -0.002396       6.03  0.0   
1                  69.75280              11.018500  -0.000139       6.03  0.0   
2                  59.35190               1.4049

,P-value,Odds ratio,Thresholds,F1 scores
ACTN2,1.000000e+00,4.444509e+06,2,0.002151
ALPK3,1.000000e+00,1.224834e+07,2,0.020339
FLNC,9.406915e-07,6.811688e+00,83,0.000000
MYBPC3,1.000000e+00,3.901183e+07,11,0.144796
MYH6,1.676823e-07,4.465473e+01,99,0.000000
MYH7,2.688528e-06,1.490769e+01,99,0.057762
PTPN11,2.679943e-01,1.025288e+01,54,0.150510
TNNI3,1.000000e+00,5.215107e+06,14,0.523810
TTR,1.000000e+00,6.478141e+06,28,0.629442


#### Convert highest F1 scores to dataframe

In [ ]:
updated_p_dict, hazard_ratios, f1_scores = find_threshold_vals(thresholds_by_f1_score)
p_vals = pd.DataFrame.from_dict(updated_p_dict, orient = 'index')
p_vals.columns = ["P-value"]
hazard_ratios = pd.DataFrame.from_dict(hazard_ratios, orient = "index")
hazard_ratios.columns = ["Odds ratio"]
f1_scores = pd.DataFrame.from_dict(f1_scores, orient = 'index')
f1_scores.columns = ["F1 scores"]
thresholds = pd.DataFrame.from_dict(thresholds_by_f1_score, orient = 'index')
thresholds.columns = ["Thresholds"]
df = p_vals.join(hazard_ratios).join(thresholds).join(f1_scores)
df

ACTN2
ALPK3
FLNC
MYBPC3
MYH6
MYH7


KeyboardInterrupt: 